# Step 2-0. 로컬 임베딩 모델을 사용한 벡터 데이터 업로드
Sentence Transformers 모델로 텍스트를 벡터로 변환하여 OpenSearch에 업로드합니다.

## 배경 지식

### 벡터(Vector)와 임베딩(Embedding)
- **벡터**: 숫자의 목록(리스트). 어떤 개념이나 의미를 수치로 표현한 것
- **word2vec (= embedding)**: 단어가 다르더라도 **의미가 비슷한 것**을 찾을 수 있음
  - 예: "사람" [0, 0, 1] vs "그러고도 니가 사람이야?" [0, 0, 0.1] → 벡터 방향이 같으면 의미가 비슷
  - 반면: "사람" [0, 0, 1] vs "바나나" [1, 0, 0] → 벡터 방향이 다르면 의미도 다름

### Vector Search
텍스트나 이미지 등을 벡터로 변환한 후, 유사한 벡터를 검색하여 **의미 기반**으로 결과를 찾는 기술
- **코사인 유사도**: 벡터 간 각도를 이용해 유사도 측정 (방향이 같으면 유사)
- **유클리드 거리**: 벡터 간 거리로 유사도 측정
- BM25(키워드 기반)와의 차이: Vector Search는 **"의미 기반 검색"**

### k-NN (k-Nearest Neighbors)
새로운 데이터가 주어졌을 때, 주변에 있는 **k개의 이웃**을 보고 판단하는 알고리즘
- **k**: 주변 이웃 개수 (작으면 민감, 크면 덜 민감)
- **distance metric**: 거리 계산 방법 (유클리드, 코사인 등)

In [ ]:
!pip install -q boto3==1.38.46 opensearch-py==2.8.0 sentence-transformers==4.1.0

## 1. 설정 (Configuration)

In [ ]:
import os, json

# Step 0에서 저장한 설정 불러오기
try:
    with open("../config.json") as f:
        _config = json.load(f)
    print("✅ config.json 로드 완료")
except FileNotFoundError:
    raise FileNotFoundError("❌ config.json을 찾을 수 없습니다. Step 0 노트북을 먼저 실행해주세요.")

HOST = _config.get("OPENSEARCH_HOST")
if not HOST:
    raise ValueError("❌ config.json에 OPENSEARCH_HOST 값이 없습니다. Step 0 노트북을 먼저 실행해주세요.")
DEFAULT_REGION = _config.get("DEFAULT_REGION", "ap-northeast-2")
PROFILE = _config.get("PROFILE", "skku-opensearch-session")

## 2. OpenSearch 클라이언트 생성

In [4]:
import boto3
from opensearchpy import OpenSearch, AWSV4SignerAuth, RequestsHttpConnection

service = 'aoss'
credentials = boto3.Session(profile_name=PROFILE).get_credentials()
auth = AWSV4SignerAuth(credentials, DEFAULT_REGION, service)

client = OpenSearch(
    hosts=[{'host': HOST, 'port': 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

print("OpenSearch 클라이언트 생성 완료")

OpenSearch 클라이언트 생성 완료


## 3. 임베딩 모델 로드
한국어 포함 다국어에 성능이 좋은 Sentence Transformer 모델을 로드합니다.
처음 실행 시 모델 다운로드가 필요하여 시간이 걸릴 수 있습니다.

In [5]:
from sentence_transformers import SentenceTransformer

embedding_model_name = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
vector_dimension = 384  # 이 모델의 벡터 차원 수

print(f"Loading embedding model '{embedding_model_name}'...")
model = SentenceTransformer(embedding_model_name)
print("Embedding model loaded successfully.")

/Users/gnidoc327/ws/side/skku-kdt-opensearch-25/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading embedding model 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'...
Embedding model loaded successfully.


## 4. 벡터 인덱스 생성
k-NN 벡터 검색을 위한 인덱스 설정 및 매핑을 정의합니다.

In [6]:
from time import sleep

index_name = 'vector-test'

index_body = {
    "settings": {
        "index": {
            "knn": True,
            "knn.algo_param.ef_search": 100,
            "analysis": {
                "analyzer": {
                    "korean_nori_analyzer": {
                        "type": "custom",
                        "tokenizer": "nori_tokenizer",
                        "filter": [
                            "nori_part_of_speech",
                            "nori_readingform",
                            "lowercase"
                        ]
                    }
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "content_vector": {
                "type": "knn_vector",
                "dimension": vector_dimension,
                "method": {
                    "name": "hnsw",
                    "space_type": "cosinesimil",
                    "engine": "nmslib"
                }
            },
            "post_id": {"type": "integer"},
            "title": {"type": "text", "analyzer": "korean_nori_analyzer"},
            "content": {"type": "text", "analyzer": "korean_nori_analyzer"},
            "author": {"type": "keyword"},
            "category": {"type": "keyword"},
            "tags": {"type": "keyword"},
            "created_at": {"type": "date"}
        }
    }
}

if not client.indices.exists(index=index_name):
    print(f"Creating new index '{index_name}'...")
    response = client.indices.create(index=index_name, body=index_body)
    print(f"Index '{index_name}' created successfully.")
    sleep(5)
else:
    print(f"Index '{index_name}' already exists.")

Creating new index 'vector-test'...
Index 'vector-test' created successfully.


## 5. 데이터 임베딩 및 업로드
JSON 데이터를 로드하고, 각 문서의 제목+내용을 벡터로 변환하여 Bulk API로 업로드합니다.

In [7]:
import json
from opensearchpy import helpers

json_file_path = '../../data/json_data.json'

with open(json_file_path, 'r', encoding='utf-8') as f:
    documents = json.load(f)
print(f"Successfully loaded {len(documents)} documents.")

def generate_bulk_actions(docs, idx_name):
    for doc in docs:
        text_to_embed = f"{doc.get('title', '')}\n{doc.get('content', '')}"
        vector = model.encode(text_to_embed).tolist()

        source_data = {
            "content_vector": vector,
            "post_id": doc.get("post_id"),
            "title": doc.get("title"),
            "content": doc.get("content"),
            "author": doc.get("author"),
            "category": doc.get("category"),
            "tags": doc.get("tags"),
            "created_at": doc.get("created_at")
        }

        yield {
            "_index": idx_name,
            "_source": source_data
        }

print("Starting data embedding and uploading...")
success, failed = helpers.bulk(client, generate_bulk_actions(documents, index_name))

print(f"Successfully indexed {success} documents.")
if failed:
    print(f"Failed to index {len(failed)} documents.")
    for i, item in enumerate(failed[:5]):
        print(f"Failed item {i+1}: {item}")

Successfully loaded 50 documents.
Starting data embedding and uploading...
Successfully indexed 50 documents.
